In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import colorsys
import numpy as np
import xml.etree.ElementTree as ET
from PIL import Image
import csv
from pathlib import Path
import re
import pandas as pd
import os
import pickle

# **Return list of new solution**

In [ ]:
#filter the the lowest consumption and median of all ff
def getNewColors(path):
    #read csv
    df = pd.read_csv(path)
    #list of solution
    solution=[]
    for row in tuple(df.solution):
      solution.append(eval(row))
    #create list fitness function
    ff=[]
    for row in tuple(df['fitness function']):
      ff.append(eval(row))
    #3 ff and split
    consum=[]
    contr=[]
    original=[]
    for row in ff:
      consum.append(row[0])
      contr.append(abs(row[1]))
      original.append(row[2])
    new_interfaces_index=[]
    #min consump
    new_interfaces_index.append(solution[np.argmin(consum)])
    #median consump
    new_interfaces_index.append(solution[np.argsort(consum)[len(consum)//2]])
    #median contrast
    new_interfaces_index.append(solution[np.argmax(contr)])
    #new_interfaces_index.append(solution[np.argsort(contr)[len(contr)//2]])
    #median original
    #new_interfaces_index.append(solution[np.argsort(original)[len(original)//2]])
    new_interfaces_index.append(solution[np.argmin(original)])
    return new_interfaces_index

In [ ]:
# function to take all dir
def getdir(path):
    return sorted(os.listdir(path))

# function to take all GUI's
def getSnapshot(path):
    data = []
    for filename in sorted(os.listdir(path)):
        if filename.endswith("uix"):
            tree = ET.parse(path + "/" + filename)
            # getting the parent tag of the xml document
            root = tree.getroot()
            data.append(root)
    return data
    
# function to take all GUI's
def getScreenshot(path):
    data = []
    for filename in sorted(os.listdir(path)):
        if filename.endswith("png"):
            data.append(path + "/" + filename)
    return data


# create a list to get all components and call the getComponents function
def getComponentsFromSnapshot(root):
    lista = []
    for comp in root.iter():
        lista.append(comp)
    return lista


def getPixels(c, tipo):
    # takes ranges based on type and split it
    values = re.findall(r'\d+', c[16][1])
    component = c[3][1]
    if tipo == 1:
        values = re.findall(r'\d+', c[17][1])
        component = c[4][1]
    asse_x_inizio = int(values[0])
    asse_y_inizio = int(values[1])
    asse_x_fine = int(values[2])
    asse_y_fine = int(values[3])
    coordinate = (asse_x_inizio, asse_y_inizio, asse_x_fine, asse_y_fine)
    return coordinate


def getView(root):
    C = getComponentsFromSnapshot(root)
    list_view = []
    # iterate all component in bottom up
    for c in C:
        c_item = c.items()
        if len(c_item) == 17 and c_item[3][1].find("ImageView") != -1 and c_item[3][1].find("ImageButton") != -1:
            list_view.append(getPixels(c_item, 0))
        elif len(c_item) == 18 and c_item[4][1].find("ImageView") != -1 and c_item[4][1].find("ImageButton") != -1:
            list_view.append(getPixels(c_item, 1))
    return list_view


# check index color
def IndexColor(img, i, j, old_color):
    for k, color in enumerate(old_color):
        data = img.getpixel((i, j))
        if (data[0] == color[0] and data[1] == color[1] and data[2] == color[2]):
            return k
    return None


# check imageview inside list
def checkImageView(view_list, i, j):
    for n, view in enumerate(view_list):
        if (i >= view[0] and i <= view[2] and j >= view[1] and j <= view[3]) == True:
            return True
    return False


# creates the new interfaces based on the generated colors
def buildImage(img_path, snap_path, name_snap, old_color, new_color, h, t):
    img = Image.open(img_path)
    width = img.size[0]
    height = img.size[1]
    view_list = getView(snap_path)
    for i in range(0, width):  # process all pixels
        for j in range(0, height):
            if len(view_list) > 0:
                c_images = checkImageView(view_list, i, j)
                if not c_images:
                    k = IndexColor(img, i, j, old_color)
                    if k is not None:
                        img.putpixel((i, j), (new_color[k][0], new_color[k][1], new_color[k][2]))
            else:
                k = IndexColor(img, i, j, old_color)
                if k is not None:
                    img.putpixel((i, j), (new_color[k][0], new_color[k][1], new_color[k][2]))
    tipo=""
    if t==0:
      tipo="min_cons"
    elif t==1:
      tipo="median_cons"
    elif t==2:
      tipo="max_contr"
    else:
      tipo="min_orig"
    # creation new interfaces
    new_interface = "/content/gdrive/Shareddrives/Tesi di Laurea Magistrale/Risultati/" + name_snap + "_NSGA2/" + name_snap + "_" + tipo + "_" + str(h) + "_new_NSGA2.png"
    img.save(new_interface)

# **MAIN**

In [ ]:
dir_path="Snapshot"
momentan=0
for dir in getdir(dir_path):
    print(dir)
  #if momentan>8:
    new_interfaces_index = getNewColors("Risultati/Data-"+dir+"-NSGA2.csv")
    new_interface_path = "Risultati/" + dir + "_NSGA2"
    # if dir doesn't exist will create a new dir
    Path(new_interface_path).mkdir(parents=True, exist_ok=True)
    bocc_path = "Dati/" + dir + "_BOCP.pickle"
    with open(bocc_path, 'rb') as handle:
      BOCP = pickle.load(handle)
    old_colors=list(BOCP.keys())
    # get snapshot
    list_snapshot = getSnapshot(dir_path +"/"+ dir)
    list_images= getScreenshot(dir_path +"/"+ dir)
    for t, new_colors in enumerate(new_interfaces_index):
      for h, image in enumerate(list_images):
        print(t)
        buildImage(image, list_snapshot[h], dir, old_colors, new_colors, h, t)
  #momentan+=1


# **Calculated Consumption**

In [ ]:
def PowerPixel(color):
    return (color[0] * 76.23 + 0.23) + (color[1] * 89.97 + 2.34) + (color[2] * 148.40 + 7.30)
def TotalPower(img_path):
    img = Image.open(img_path)
    width = img.size[0]
    height = img.size[1]
    totalPower=0
    for i in range(0, width):  # process all pixels
        for j in range(0, height):
          color = img.getpixel((i, j))
          totalPower+=PowerPixel(color)
    return totalPower

# function to take all dir
def getfolder(path):
  data = []
  for filename in sorted(os.listdir(path)):
        if filename.endswith("_NSGA2"):
          data.append(filename)
  return data

In [ ]:
dir_path="Risultati"
header_app = ['app_name', 'Max_Contrast', 'Median_Consum','Min_Consum','Min_Original','Original_Consumption']
original_dir=getdir("Snapshot")
with open('Risultati/test-consumption-NSGA2.csv','w') as f2:
      writer = csv.writer(f2)
      # write the header
      writer.writerow(header_app)
      for h,dir in enumerate(getfolder(dir_path)):
        print(dir)
        # write the data
        list_images=getScreenshot(dir_path +"/"+ dir)
        i=0
        totConsum=0
        data=[]
        data.append(dir)
        for images in list_images:
          if i==2:
            totConsum+=TotalPower(images)
            data.append(totConsum)
            totConsum=0
            i=0
          else:
            totConsum+=TotalPower(images)
            i+=1
        list_images=getScreenshot("Snapshot/"+original_dir[h])
        print(totConsum) 
        for images in list_images:
          print(images)
          totConsum+=TotalPower(images)
        data.append(totConsum)
        writer.writerow(data)